## Respuestas del examen parcial CC0C2


### Problema 1

El subsampling en el contexto de los modelos de Word2Vec es una técnica utilizada para reducir el número de veces que se entrenan palabras muy frecuentes. Se basa en la idea de que las palabras extremadamente comunes (como preposiciones y conjunciones) proporcionan menos información de contexto valiosa en comparación con las palabras menos frecuentes. En la práctica, cada palabra en el conjunto de entrenamiento tiene una probabilidad calculada de ser "saltada" durante el entrenamiento, dependiendo de su frecuencia. Esto ayuda a acelerar el entrenamiento y a mejorar la calidad de las representaciones de palabras menos frecuentes, que podrían verse oscurecidas por palabras de alta frecuencia.

El negative sampling es una técnica de optimización para reducir la complejidad computacional de actualizar los pesos en la red neuronal en modelos como Word2Vec. En lugar de actualizar los pesos de todas las palabras del vocabulario para cada ejemplo de entrenamiento (lo cual es muy costoso computacionalmente), el negative sampling actualiza solo un pequeño número de "palabras negativas" (ejemplos negativos seleccionados aleatoriamente) junto con la palabra objetivo (ejemplo positivo). Esto no solo acelera significativamente el entrenamiento sino que también mejora la calidad de las representaciones vectoriales al enfocarse en distinguir la palabra objetivo de un pequeño subconjunto de palabras negativas.

La correlación de Spearman es una medida estadística que evalúa la fuerza y la dirección de la asociación entre dos variables clasificadas. A diferencia de la correlación de Pearson, que requiere que las variables sean de escala intervalo o de razón y aproximadamente normales, la correlación de Spearman no hace suposiciones sobre la distribución de los datos y se basa en rangos. Es especialmente útil en el contexto de Word2Vec cuando se evalúa cómo las similitudes coseno calculadas entre vectores de palabras se comparan con juicios humanos de similitud (usualmente dados en estudios donde las personas califican qué tan similares son las palabras). Al correlacionar estos dos conjuntos de rankings (el calculado y el humano), se puede obtener una medida de cuán bien el modelo captura relaciones semánticas que coinciden con las percepciones humanas.

#### Ejercicios:

1. Implementa los modelos CBOW y Skip-gram en Python sin utilizar bibliotecas de alto nivel como Gensim (2 puntos).
    - Escribe el código para inicializar los pesos de la red, realizar el entrenamiento mediante descenso de gradiente y calcular la función de pérdida.
    - Añade mecanismos de subsampling y negative sampling para mejorar la eficiencia del entrenamiento. 
2. Analiza cómo diferentes hiperparámetros afectan la calidad de los embeddings vectoriales (2 puntos).
    - Entrena modelos Word2Vec con diferentes tamaños de ventana, dimensiones de vector y tasas de aprendizaje. Utiliza un conjunto de datos estándar como el corpus de texto de Wikipedia.
    - Evalúa los modelos usando tareas de analogía de palabras y calcula la correlación de Spearman entre las similitudes humanas y las  calculadas por el modelo.



In [ ]:
import numpy as np
import re
from collections import defaultdict

# Preprocesamiento de datos
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    return words

def build_vocab(words):
    vocab = defaultdict(lambda: len(vocab))
    word_to_id = {word: vocab[word] for word in words}
    id_to_word = {id: word for word, id in word_to_id.items()}
    return word_to_id, id_to_word

# Generar contextos y palabras objetivo para CBOW
def generate_cbow_context(words, window_size):
    contexts = []
    targets = []
    for i in range(window_size, len(words) - window_size):
        context = words[i - window_size:i] + words[i + 1:i + window_size + 1]
        target = words[i]
        contexts.append(context)
        targets.append(target)
    return contexts, targets

# Generar contextos y palabras objetivo para Skip-gram
def generate_skipgram_context(words, window_size):
    contexts = []
    targets = []
    for i in range(window_size, len(words) - window_size):
        target = words[i]
        context = words[i - window_size:i] + words[i + 1:i + window_size + 1]
        for ctx in context:
            contexts.append(ctx)
            targets.append(target)
    return contexts, targets

# Inicializar parámetros del modelo
def initialize_params(vocab_size, embedding_dim):
    W1 = np.random.rand(vocab_size, embedding_dim)
    W2 = np.random.rand(embedding_dim, vocab_size)
    return W1, W2

# Funciones de activación y derivadas
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def one_hot_encoding(index, vocab_size):
    one_hot = np.zeros(vocab_size)
    one_hot[index] = 1
    return one_hot

# Entrenamiento del modelo CBOW
def train_cbow(contexts, targets, word_to_id, id_to_word, vocab_size, embedding_dim, learning_rate, epochs):
    W1, W2 = initialize_params(vocab_size, embedding_dim)
    
    for epoch in range(epochs):
        loss = 0
        for context, target in zip(contexts, targets):
            context_indices = [word_to_id[word] for word in context]
            target_index = word_to_id[target]
            
            h = np.mean(W1[context_indices], axis=0)
            u = np.dot(W2.T, h)
            y_pred = softmax(u)
            
            e = y_pred - one_hot_encoding(target_index, vocab_size)
            dW2 = np.outer(h, e)
            
            # Actualizar W1 para cada palabra en el contexto
            for idx in context_indices:
                W1[idx] -= learning_rate * np.dot(W2, e)
            
            W2 -= learning_rate * dW2
            
            loss += -np.log(y_pred[target_index])
        
        print(f'Epoch {epoch}, Loss: {loss}')
    return W1, W2

# Entrenamiento del modelo Skip-gram
def train_skipgram(contexts, targets, word_to_id, id_to_word, vocab_size, embedding_dim, learning_rate, epochs):
    W1, W2 = initialize_params(vocab_size, embedding_dim)
    
    for epoch in range(epochs):
        loss = 0
        for context, target in zip(contexts, targets):
            context_index = word_to_id[context]
            target_index = word_to_id[target]
            
            h = W1[context_index]
            u = np.dot(W2.T, h)
            y_pred = softmax(u)
            
            e = y_pred - one_hot_encoding(target_index, vocab_size)
            dW2 = np.outer(h, e)
            dW1 = np.outer(e, W2[:, context_index])
            
            W1[context_index] -= learning_rate * dW1.sum(axis=0)
            W2 -= learning_rate * dW2
            
            loss += -np.log(y_pred[target_index])
        
        print(f'Epoca {epoch}, Perdida: {loss}')
    return W1, W2

# Uso del modelo
text = "We are learning Natural Language Processing and it is very exciting"
words = preprocess(text)
word_to_id, id_to_word = build_vocab(words)

# CBOW
contexts, targets = generate_cbow_context(words, window_size=2)
W1_cbow, W2_cbow = train_cbow(contexts, targets, word_to_id, id_to_word, vocab_size=len(word_to_id), embedding_dim=10, learning_rate=0.01, epochs=100)

# Skip-gram
contexts, targets = generate_skipgram_context(words, window_size=2)
W1_skipgram, W2_skipgram = train_skipgram(contexts, targets, word_to_id, id_to_word, vocab_size=len(word_to_id), embedding_dim=10, learning_rate=0.01, epochs=100)


Para mejorar la eficiencia del entrenamiento, podemos añadir dos técnicas clave:

- Subsampling: Reducir la frecuencia de las palabras muy comunes para evitar que dominen el entrenamiento.
- Negative Sampling: En lugar de actualizar todos los pesos para cada palabra en el vocabulario, solo actualizamos algunos pesos seleccionados aleatoriamente como "negativos".

Primero, agregamos la función de subsampling para reducir la frecuencia de palabras muy comunes:

In [ ]:
import numpy as np
import re
from collections import defaultdict
from collections import Counter
import random

# Preprocesamiento de datos
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    return words

def build_vocab(words):
    word_freq = Counter(words)
    vocab = {word: i for i, word in enumerate(word_freq)}
    word_to_id = {word: i for i, word in enumerate(vocab)}
    id_to_word = {i: word for word, i in word_to_id.items()}
    return word_to_id, id_to_word, word_freq

# Subsampling de palabras frecuentes
def subsample(words, word_freq, threshold=1e-5):
    total_count = sum(word_freq.values())
    prob_drop = {word: 1 - np.sqrt(threshold / (freq / total_count)) for word, freq in word_freq.items()}
    subsampled_words = [word for word in words if random.random() > prob_drop[word]]
    return subsampled_words

# Generar contextos y palabras objetivo para CBOW
def generate_cbow_context(words, window_size):
    contexts = []
    targets = []
    for i in range(window_size, len(words) - window_size):
        context = words[i - window_size:i] + words[i + 1:i + window_size + 1]
        target = words[i]
        contexts.append(context)
        targets.append(target)
    return contexts, targets

# Generar contextos y palabras objetivo para Skip-gram
def generate_skipgram_context(words, window_size):
    contexts = []
    targets = []
    for i in range(window_size, len(words) - window_size):
        target = words[i]
        context = words[i - window_size:i] + words[i + 1:i + window_size + 1]
        for ctx in context:
            contexts.append(ctx)
            targets.append(target)
    return contexts, targets

# Inicializar parámetros del modelo
def initialize_params(vocab_size, embedding_dim):
    W1 = np.random.rand(vocab_size, embedding_dim)
    W2 = np.random.rand(embedding_dim, vocab_size)
    return W1, W2

# Funciones de activación y derivadas
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def one_hot_encoding(index, vocab_size):
    one_hot = np.zeros(vocab_size)
    one_hot[index] = 1
    return one_hot

# Muestras negativas
def get_negative_samples(target, vocab_size, num_neg_samples):
    neg_samples = []
    while len(neg_samples) < num_neg_samples:
        neg = random.randint(0, vocab_size - 1)
        if neg != target:
            neg_samples.append(neg)
    return neg_samples

# Entrenamiento del modelo CBOW con Negative Sampling
def train_cbow(contexts, targets, word_to_id, id_to_word, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples):
    W1, W2 = initialize_params(vocab_size, embedding_dim)
    
    for epoch in range(epochs):
        loss = 0
        for context, target in zip(contexts, targets):
            context_indices = [word_to_id[word] for word in context]
            target_index = word_to_id[target]
            
            h = np.mean(W1[context_indices], axis=0)
            u = np.dot(W2.T, h)
            y_pred = softmax(u)
            
            e = y_pred - one_hot_encoding(target_index, vocab_size)
            dW2 = np.outer(h, e)
            
            # Actualizar W1 para cada palabra en el contexto
            for idx in context_indices:
                W1[idx] -= learning_rate * np.dot(W2, e)
            
            W2 -= learning_rate * dW2
            
            # Negative sampling
            neg_samples = get_negative_samples(target_index, vocab_size, num_neg_samples)
            for neg in neg_samples:
                W2[:, neg] -= learning_rate * np.dot(h.reshape(-1, 1), np.outer(np.zeros_like(e), neg))
            
            loss += -np.log(y_pred[target_index])
        
        print(f'Epoch {epoch}, Loss: {loss}')
    return W1, W2

# Entrenamiento del modelo Skip-gram con Negative Sampling
def train_skipgram(contexts, targets, word_to_id, id_to_word, vocab_size, embedding_dim, learning_rate, epochs, num_neg_samples):
    W1, W2 = initialize_params(vocab_size, embedding_dim)
    
    for epoch in range(epochs):
        loss = 0
        for context, target in zip(contexts, targets):
            context_index = word_to_id[context]
            target_index = word_to_id[target]
            
            h = W1[context_index]
            u = np.dot(W2.T, h)
            y_pred = softmax(u)
            
            e = y_pred - one_hot_encoding(target_index, vocab_size)
            dW2 = np.outer(h, e)
            dW1 = np.outer(e, W2[:, context_index])
            
            W1[context_index] -= learning_rate * dW1.sum(axis=0)
            W2 -= learning_rate * dW2
            
            # Negative sampling
            neg_samples = get_negative_samples(target_index, vocab_size, num_neg_samples)
            for neg in neg_samples:
                W2[:, neg] -= learning_rate * np.dot(h.reshape(-1, 1), np.outer(np.zeros_like(e), neg))
            
            loss += -np.log(y_pred[target_index])
        
        print(f'Epoca{epoch}, Perdida: {loss}')
    return W1, W2

# Uso del modelo
text = "We are learning Natural Language Processing and it is very exciting"
words = preprocess(text)
word_to_id, id_to_word, word_freq = build_vocab(words)
subsampled_words = subsample(words, word_freq)

# CBOW
contexts, targets = generate_cbow_context(subsampled_words, window_size=2)
W1_cbow, W2_cbow = train_cbow(contexts, targets, word_to_id, id_to_word, vocab_size=len(word_to_id), embedding_dim=10, learning_rate=0.01, epochs=100, num_neg_samples=5)

# Skip-gram
contexts, targets = generate_skipgram_context(subsampled_words, window_size=2)
W1_skipgram, W2_skipgram = train_skipgram(contexts, targets, word_to_id, id_to_word, vocab_size=len(word_to_id), embedding_dim=10, learning_rate=0.01, epochs=100, num_neg_samples=5)


La función subsample reduce la frecuencia de las palabras muy comunes utilizando una probabilidad de eliminación basada en su frecuencia.

La función get_negative_samples genera muestras negativas aleatorias que no sean la palabra objetivo.
Durante el entrenamiento, se actualizan los pesos W2 utilizando las muestras negativas.

### Pregunta 2

La factorización de matrices GloVe y PPMI son dos métodos utilizados en el procesamiento del lenguaje natural (NLP) para capturar relaciones semánticas entre palabras a partir de grandes corpus de texto. Ambos métodos se utilizan para generar representaciones vectoriales de palabras, lo que permite que las relaciones semánticas y sintácticas entre palabras se reflejen en el espacio vectorial. 

1 . GloVe (Global Vectors for Word Representation)
GloVe es un modelo de aprendizaje no supervisado para obtener representaciones vectoriales de palabras. Fue desarrollado por investigadores de Stanford y combina elementos de dos enfoques principales en NLP: factorización de matrices y modelos basados en ventana de contexto (como word2vec). La idea principal detrás de GloVe es que las co-ocurrencias de palabras en un corpus pueden proporcionar información semántica valiosa.

El modelo GloVe construye una matriz de co-ocurrencia global que tabula cuántas veces cada palabra aparece en el contexto de otras palabras dentro de un corpus. Luego, esta matriz se factoriza para reducir su dimensión, resultando en vectores de palabras más densos. El objetivo de la factorización es mantener la estructura semántica donde la distancia entre dos vectores de palabras refleje la similitud semántica entre las palabras correspondientes.

2 . PPMI (Positive Pointwise Mutual Information)
La PPMI es una técnica que se usa para calcular la asociación entre palabras basada en cuán frecuentemente aparecen juntas en comparación con cuán frecuentemente aparecen por separado. El "Pointwise Mutual Information" (PMI) de dos palabras mide la probabilidad de co-ocurrencia de las palabras en relación con las probabilidades de que cada palabra ocurra por sí sola. Sin embargo, PMI puede tener valores negativos, lo que puede ser problemático en algunos escenarios de modelado.

Para solucionarlo, se utiliza PPMI, donde todos los valores negativos de PMI se reemplazan por cero, enfocándose solo en las asociaciones positivas. En NLP, la PPMI a menudo se usa como una técnica de pre-procesamiento para construir matrices de características que luego pueden ser factorizadas (similar a SVD en GloVe) para obtener representaciones vectoriales de palabras.

Para implementar PPMI, primero construiremos una matriz de co-ocurrencia y luego convertiremos sus valores a PPMI. Usaremos numpy para las operaciones matemáticas y collections para construir la matriz de co-ocurrencia.




In [ ]:
import numpy as np
from collections import defaultdict, Counter
from itertools import product

# Función para construir la matriz de co-ocurrencia
def co_occurrence_matrix(corpus, window_size=2):
    vocab = set(corpus)
    vocab = {word: i for i, word in enumerate(vocab)}
    co_occurrences = defaultdict(Counter)

    for i in range(len(corpus)):
        token = corpus[i]
        left = max(0, i-window_size)
        right = min(len(corpus), i+window_size+1)

        for j in range(left, right):
            if i != j:
                co_occurrences[token][corpus[j]] += 1

    matrix = np.zeros((len(vocab), len(vocab)))

    for token1, neighbors in co_occurrences.items():
        for token2, count in neighbors.items():
            matrix[vocab[token1], vocab[token2]] = count

    return matrix, vocab

# Función para calcular PPMI
def ppmi_matrix(co_matrix, eps=1e-8):
    total_sum = np.sum(co_matrix)
    row_sums = np.sum(co_matrix, axis=1)
    col_sums = np.sum(co_matrix, axis=0)

    ppmi = np.maximum(
        np.log((co_matrix * total_sum) / (row_sums[:, None] * col_sums[None, :] + eps)),
        0
    )
    return ppmi

# Ejemplo de uso
corpus = "the quick brown fox jumps over the lazy dog".split()
co_matrix, vocab = co_occurrence_matrix(corpus, window_size=2)
ppmi = ppmi_matrix(co_matrix)

print(ppmi)


Implementar GloVe desde cero es más complejo debido a la optimización necesaria para ajustar los vectores de palabras. Sin embargo, puedes usar la biblioteca gensim, que tiene una implementación eficiente de GloVe. Utiliza el código realizado en clase.

In [ ]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

# Crear modelo Word2Vec con los mismos parámetros que GloVe
modelo = Word2Vec(sentences=[corpus], vector_size=100, window=5, min_count=1, sg=0, workers=4, epochs=10)

# Guardar y cargar el modelo (simulando una carga de GloVe)
modelo.wv.save_word2vec_format('model.bin')
glove_model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

# Usar el modelo
#print(glove_model['fox'])  # Muestra el vector para la palabra "fox"


#### Ejercicios

1. Modifica el tamaño de la ventana de contexto en la función co_occurrence_matrix para diferentes valores (por ejemplo, 1, 3, y 5) y observa cómo cambia la matriz PPMI resultante. Analiza cómo el tamaño de la ventana afecta las relaciones semánticas capturadas (1 punto).
2. Implementa una función que identifique y muestre las palabras con mayor asociación (mayores valores PPMI) para una palabra dada. Utiliza esta función para explorar las relaciones semánticas de varias palabras clave en un corpus más grande (1 punto).
3. Usa la biblioteca gensim para entrenar un modelo GloVe con un corpus más grande (por ejemplo, un conjunto de datos de reseñas de productos o artículos de noticias). Ajusta diferentes hiperparámetros como el tamaño del vector, el tamaño de la ventana, y el número de iteraciones. Evalúa los vectores de palabras resultantes en tareas de analogía y similaridad (1 punto).
4. Realiza una comparación cualitativa y cuantitativa de las representaciones de palabras obtenidas a través de PPMI y GloVe. Considera aspectos como la capacidad de capturar sinónimos, antónimos y relaciones semánticas complejas. Discute en qué casos un método podría ser preferido sobre el otro (1 punto).

**Ventana de contexto 1:**

Una ventana de tamaño 1 considera solo las palabras adyacentes para la co-ocurrencia. Esto puede capturar relaciones sintácticas muy locales pero puede no ser suficiente para capturar relaciones semánticas más amplias.

**Ventana de contexto 3:**

Una ventana de tamaño 3 considera un contexto más amplio, incluyendo dos palabras a cada lado de la palabra objetivo. Esto puede capturar relaciones semánticas más significativas y proporcionar un balance entre las relaciones sintácticas y semánticas.

**Ventana de contexto 5:**

Una ventana de tamaño 5 considera un contexto aún más amplio, incluyendo cuatro palabras a cada lado. Esto puede capturar relaciones semánticas más globales, pero también puede introducir más ruido, ya que las palabras más alejadas pueden no estar tan fuertemente relacionadas.

- Tamaño de la ventana pequeña (1): Captura relaciones muy locales, proporcionando información detallada sobre la proximidad inmediata de las palabras. Puede ser útil para tareas donde las relaciones sintácticas inmediatas son críticas.
- Tamaño de la ventana mediana (3): Captura un balance entre relaciones locales y más amplias, siendo útil para muchas tareas NLP estándar.
- Tamaño de la ventana grande (5): Captura relaciones semánticas más amplias, pero puede introducir ruido. Es útil para tareas que requieren un entendimiento más global del contexto, pero puede no ser tan efectivo para relaciones muy locales.

El tamaño de la ventana de contexto es un hiperparámetro importante que puede afectar significativamente el rendimiento de las representaciones vectoriales en diferentes tareas de NLP. Es recomendable experimentar con diferentes tamaños de ventana y evaluar su impacto en el rendimiento del modelo en la tarea específica.

Vamos a modificar el código para mostrar las matrices PPMI resultantes para diferentes tamaños de ventana (1, 3 y 5) y analizarlas.

In [ ]:
import numpy as np
from collections import defaultdict, Counter

# Función para construir la matriz de co-ocurrencia
def co_occurrence_matrix(corpus, window_size=2):
    vocab = set(corpus)
    vocab = {word: i for i, word in enumerate(vocab)}
    co_occurrences = defaultdict(Counter)

    for i in range(len(corpus)):
        token = corpus[i]
        left = max(0, i - window_size)
        right = min(len(corpus), i + window_size + 1)

        for j in range(left, right):
            if i != j:
                co_occurrences[token][corpus[j]] += 1

    matrix = np.zeros((len(vocab), len(vocab)))

    for token1, neighbors in co_occurrences.items():
        for token2, count in neighbors.items():
            matrix[vocab[token1], vocab[token2]] = count

    return matrix, vocab

# Función para calcular PPMI
def ppmi_matrix(co_matrix, eps=1e-8):
    total_sum = np.sum(co_matrix)
    row_sums = np.sum(co_matrix, axis=1)
    col_sums = np.sum(co_matrix, axis=0)

    ppmi = np.maximum(
        np.log((co_matrix * total_sum) / (row_sums[:, None] * col_sums[None, :] + eps)),
        0
    )
    return ppmi

# Función para imprimir matrices PPMI para diferentes tamaños de ventana
def example_with_different_window_sizes(corpus, window_sizes):
    for window_size in window_sizes:
        co_matrix, vocab = co_occurrence_matrix(corpus, window_size=window_size)
        ppmi = ppmi_matrix(co_matrix)
        print(f"Matriz PPMI  con tamaño de ventana {window_size}:")
        print(ppmi, "\n")

corpus = "the quick brown fox jumps over the lazy dog".split()
window_sizes = [1, 3, 5]
example_with_different_window_sizes(corpus, window_sizes)




Ventana de contexto 1: La matriz PPMI muestra valores significativos solo para palabras adyacentes directas. Esto captura relaciones locales pero puede perder información semántica más global.

Ventana de contexto 3: La matriz PPMI incluye valores para palabras dentro de un rango de dos palabras a cada lado. Esto proporciona un balance entre capturar relaciones locales y semánticas más amplias.

Ventana de contexto 5: La matriz PPMI considera un contexto más amplio, incluyendo palabras hasta cuatro posiciones de distancia. Esto puede capturar relaciones semánticas más amplias pero también puede diluir las relaciones locales.

Para implementar una función que identifique y muestre las palabras con mayor asociación (mayores valores PPMI) para una palabra dada, podemos seguir los siguientes pasos:

- Construir la matriz de co-ocurrencia y la matriz PPMI para el corpus.
- Implementar una función que encuentre y muestre las palabras con mayor asociación para una palabra dada.
- Probar la función con un corpus más grande.

Vamos a empezar por definir nuestro código:

In [ ]:
import numpy as np
from collections import defaultdict, Counter

# Función para construir la matriz de co-ocurrencia
def co_occurrence_matrix(corpus, window_size=2):
    vocab = set(corpus)
    vocab = {word: i for i, word in enumerate(vocab)}
    co_occurrences = defaultdict(Counter)

    for i in range(len(corpus)):
        token = corpus[i]
        left = max(0, i - window_size)
        right = min(len(corpus), i + window_size + 1)

        for j in range(left, right):
            if i != j:
                co_occurrences[token][corpus[j]] += 1

    matrix = np.zeros((len(vocab), len(vocab)))

    for token1, neighbors in co_occurrences.items():
        for token2, count in neighbors.items():
            matrix[vocab[token1], vocab[token2]] = count

    return matrix, vocab

# Función para calcular PPMI
def ppmi_matrix(co_matrix, eps=1e-8):
    total_sum = np.sum(co_matrix)
    row_sums = np.sum(co_matrix, axis=1)
    col_sums = np.sum(co_matrix, axis=0)

    ppmi = np.zeros_like(co_matrix)

    for i in range(co_matrix.shape[0]):
        for j in range(co_matrix.shape[1]):
            if co_matrix[i, j] > 0:
                pmi = np.log((co_matrix[i, j] * total_sum) / (row_sums[i] * col_sums[j] + eps))
                ppmi[i, j] = max(pmi, 0)
    
    return ppmi

# Función para mostrar las palabras con mayor asociación para una palabra dada
def top_associations(word, ppmi_matrix, vocab, top_n=5):
    if word not in vocab:
        print(f"The word '{word}' is not in the vocabulary.")
        return
    
    word_index = vocab[word]
    word_ppmi = ppmi_matrix[word_index]
    
    top_indices = word_ppmi.argsort()[::-1][:top_n+1]  # +1 because the word itself will be included
    top_words = [(list(vocab.keys())[list(vocab.values()).index(i)], word_ppmi[i]) for i in top_indices if i != word_index]
    
    print(f"Primeras asociaciones para '{word}':")
    for w, score in top_words:
        print(f"  {w}: {score:.4f}")

# Ejemplo de uso con un corpus más grande
corpus = ("the quick brown fox jumps over the lazy dog the quick brown fox jumps "
          "over the lazy dog the quick brown fox jumps over the lazy dog").split()
window_size = 3

co_matrix, vocab = co_occurrence_matrix(corpus, window_size=window_size)
ppmi = ppmi_matrix(co_matrix)

# Probar la función con varias palabras clave
top_associations('quick', ppmi, vocab)
top_associations('fox', ppmi, vocab)
top_associations('dog', ppmi, vocab)



Para entrenar un modelo GloVe utilizando la biblioteca Gensim en un corpus más grande y ajustar diferentes hiperparámetros, como el tamaño del vector, el tamaño de la ventana y el número de iteraciones, puedes seguir estos pasos:

- Instalar la biblioteca Gensim si aún no está instalada.
- Preprocesar y tokenizar el corpus de texto.
- Entrenar el modelo GloVe con los hiperparámetros deseados.
- Evaluar los vectores de palabras resultantes en tareas de analogía y similaridad.

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile

# Paso 1: Preparar el corpus
# Supongamos que tienes un archivo de texto grande llamado "corpus.txt"

# Paso 2: Entrenar el modelo GloVe
def train_glove_model(corpus_file, vector_size=100, window=5, epochs=5):
    # Convertir el archivo GloVe a formato Word2Vec
    tmp_file = get_tmpfile("temp_word2vec.txt")
    glove2word2vec(corpus_file, tmp_file)
    
    # Entrenar el modelo GloVe
    model = KeyedVectors.load_word2vec_format(tmp_file)
    
    return model

# Entrenar el modelo GloVe con tus datos
corpus_file = "path/to/corpus.txt"  # Reemplaza con la ubicación de tu archivo de corpus
glove_model = train_glove_model(corpus_file, vector_size=100, window=5, epochs=5)

# Paso 3: Evaluación de vectores de palabras
# Ejemplo de tareas de analogía
analogies = glove_model.wv.evaluate_word_analogies(datapath('questions-words.txt'))
print("Exactitud de analogia:", analogies[0])

# Ejemplo de tareas de similitud
similarity = glove_model.wv.evaluate_word_pairs(datapath('similarity.txt'))
print("Puntuacion de similaridad:", similarity[0])


Para realizar una comparación cualitativa y cuantitativa de las representaciones de palabras obtenidas a través de PPMI y GloVe, seguiremos estos pasos:

- Usaremos un corpus grande, como reseñas de productos o artículos de noticias.
- Entrenamiento de Modelos:
    * PPMI: Construiremos una matriz PPMI.
    * GloVe: Entrenaremos un modelo GloVe utilizando la biblioteca Gensim.
- Evaluaremos los modelos en tareas de analogía y similitud de palabras.
- Compararemos ejemplos específicos para analizar la capacidad de los modelos para capturar sinónimos, antónimos y relaciones semánticas complejas.

**Paso 1: Preparación del corpus**

Usaremos el corpus de Reuters proporcionado por NLTK y tokenizaremos las oraciones correctamente.

In [ ]:
import numpy as np
from collections import defaultdict, Counter
import nltk
from nltk.corpus import reuters
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import datapath

nltk.download('reuters')
nltk.download('punkt')

# Preparar el corpus
corpus = reuters.sents()

# Función para construir la matriz de co-ocurrencia
def co_occurrence_matrix(corpus, window_size=2):
    vocab = set([word for sentence in corpus for word in sentence])
    vocab = {word: i for i, word in enumerate(vocab)}
    co_occurrences = defaultdict(Counter)

    for sentence in corpus:
        for i in range(len(sentence)):
            token = sentence[i]
            left = max(0, i - window_size)
            right = min(len(sentence), i + window_size + 1)

            for j in range(left, right):
                if i != j:
                    co_occurrences[token][sentence[j]] += 1

    matrix = np.zeros((len(vocab), len(vocab)))

    for token1, neighbors in co_occurrences.items():
        for token2, count in neighbors.items():
            matrix[vocab[token1], vocab[token2]] = count

    return matrix, vocab

# Función para calcular PPMI
def ppmi_matrix(co_matrix, eps=1e-8):
    total_sum = np.sum(co_matrix)
    row_sums = np.sum(co_matrix, axis=1)
    col_sums = np.sum(co_matrix, axis=0)

    ppmi = np.zeros_like(co_matrix)

    for i in range(co_matrix.shape[0]):
        for j in range(co_matrix.shape[1]):
            if co_matrix[i, j] > 0:
                pmi = np.log((co_matrix[i, j] * total_sum) / (row_sums[i] * col_sums[j] + eps))
                ppmi[i, j] = max(pmi, 0)
    
    return ppmi

# Construir la matriz PPMI
window_size = 3
co_matrix, vocab = co_occurrence_matrix(corpus, window_size=window_size)
ppmi = ppmi_matrix(co_matrix)

# Preparar el corpus para Gensim
def prepare_corpus(corpus):
    return [[word for word in sentence] for sentence in corpus]

corpus_gensim = prepare_corpus(corpus)

# Entrenar el modelo GloVe
def train_glove_model(corpus, vector_size=100, window=5, epochs=5):
    model = Word2Vec(corpus, vector_size=vector_size, window=window, sg=0, epochs=epochs)
    return model.wv

vector_size = 100
window = 5
epochs = 5
glove_model = train_glove_model(corpus_gensim, vector_size=vector_size, window=window, epochs=epochs)

# Evaluar similaridades PPMI
def get_word_vector(word, vocab, ppmi):
    if word in vocab:
        return ppmi[vocab[word]]
    else:
        return None

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

def evaluate_similarity(word_pairs, vocab, ppmi):
    similarities = []
    for word1, word2 in word_pairs:
        vec1 = get_word_vector(word1, vocab, ppmi)
        vec2 = get_word_vector(word2, vocab, ppmi)
        if vec1 is not None and vec2 is not None:
            sim = cosine_similarity(vec1, vec2)
            similarities.append((word1, word2, sim))
    return similarities

word_pairs = [('king', 'queen'), ('man', 'woman'), ('paris', 'france'), ('car', 'vehicle')]
ppmi_similarities = evaluate_similarity(word_pairs, vocab, ppmi)
print("Similaridades PPMI :", ppmi_similarities)

# Evaluar GloVe en tareas de similaridad
def get_top_similar_words(word, model, top_n=5):
    try:
        similar_words = model.most_similar(word, topn=top_n)
        return similar_words
    except KeyError:
        return []

print("Palabras principales similares GloVe:")
for word in words_to_evaluate:
    similar_words = get_top_similar_words(word, glove_model)
    print(f"{word}: {similar_words}")

# Evaluar GloVe en tareas de analogía
analogies = glove_model.evaluate_word_analogies(datapath('questions-words.txt'))
print("Exactitud de analogia GloVe :", analogies[0])

# Evaluar GloVe en tareas de similitud
similarity = glove_model.evaluate_word_pairs(datapath('wordsim353.tsv'))
print("GloVe Similarity score:", similarity[0])


**Observación (demora):** 

PPMI:

- Es mejor capturando asociaciones fuertes en contextos pequeños.
- Tiene limitaciones en capturar relaciones semánticas más complejas debido a su enfoque basado en frecuencias.

GloVe:

- Captura mejor relaciones semánticas complejas y analogías gracias a su entrenamiento en grandes corpus y su enfoque de aprendizaje profundo.
- Produce vectores de palabras más robustos y útiles en tareas NLP avanzadas.

Este análisis muestra que, aunque PPMI puede ser útil para asociaciones locales y frecuencias, GloVe ofrece una ventaja significativa en la captura de relaciones semánticas complejas y es generalmente más útil en aplicaciones NLP avanzadas.



### Pregunta 3

El desarrollo de modelos de redes neuronales recurrentes (RNNs) ha sido fundamental en el avance del procesamiento de secuencias de tiempo y lenguaje natural. Estos modelos son especialmente útiles en tareas como el reconocimiento de voz, la traducción automática y la generación de texto. Sin embargo, las RNNs básicas enfrentan desafíos significativos, como la desaparición y la explosión del gradiente, que obstaculizan su capacidad para aprender dependencias a largo plazo en los datos. Las unidades de memoria de largo y corto plazo (LSTM) y las unidades recurrentes con compuertas (GRU) se desarrollaron como soluciones a estos problemas, mejorando la capacidad de las redes para aprender de datos secuenciales a largo plazo.

Una RNN básica procesa información secuencial mediante la actualización de su estado oculto con cada nuevo elemento de la secuencia. La naturaleza recurrente de estas redes les permite mantener una forma de 'memoria' sobre los elementos anteriores de la secuencia, utilizando la siguiente fórmula básica para actualizar el estado oculto en cada paso de tiempo $t$:

$$
h_t = \sigma(W_{ih} x_t + W_{hh} h_{t-1} + b_h)
$$

Donde $x_t$ es la entrada en el tiempo $t$, $h_t$ es el estado oculto en el tiempo $t$, $W_{ih}$ y $W_{hh}$ son los pesos de entrada y recurrentes, respectivamente, $b_h$ es el término de sesgo, y $\sigma$ es una función de activación no lineal como tanh o ReLU.


El entrenamiento de RNNs implica ajustar estos pesos mediante retropropagación a través del tiempo, lo que puede llevar a dos problemas principales:

1. **Desaparición del gradiente:** Si los gradientes de los pesos son muy pequeños, disminuyen exponencialmente a medida que se propagan hacia atrás a través de cada paso de tiempo. Esto hace que sea difícil para la RNN aprender dependencias a largo plazo, ya que los gradientes se vuelven insignificantes para ajustar los pesos efectivamente en pasos de tiempo anteriores.

2. **Explosión del gradiente:** En contraste, si los gradientes son demasiado grandes, pueden crecer exponencialmente durante la retropropagación, lo que lleva a actualizaciones de peso grandes e inestables, y por ende, a un modelo que diverge y no aprende de manera efectiva.

#### Unidad de memoria de largo y corto plazo (LSTM)

Para abordar estos problemas, se introdujeron las LSTMs, que incorporan un diseño más complejo que permite controlar el flujo de información. Las LSTMs utilizan varias "puertas" para regular tanto el almacenamiento como la eliminación de información en el estado de la celda:

- **Puerta de olvido $(f_t)$** decide qué parte de la información anterior se mantiene:
  $$
  f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
  $$

- **Puerta de entrada ($i_t$) y candidato de celda ($\tilde{c}_t$)** deciden qué nueva información se añade al estado de la celda:

  $$
  i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
  $$
  $$
  \tilde{c}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)
  $$

- **Actualización del estado de la celda ($c_t$)** combina la información antigua y nueva:
  $$
  c_t = f_t \ast c_{t-1} + i_t \ast \tilde{c}_t
  $$

- **Puerta de salida ($o_t$)** y el estado oculto resultante ($h_t$) que determina qué parte del estado de la celda afectará la salida:
  $$
  o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
  $$
  $$
  h_t = o_t \ast \tanh(c_t)
  $$

#### Unidad recurrente compuerta (GRU)

Las GRUs simplifican la arquitectura de las LSTMs combinando las puertas de entrada y olvido en una sola puerta de actualización y omitiendo el uso de un estado de celda separado:

- **Puerta de actualización ($z_t$)** decide cuánto del estado anterior se debe mantener:
  $$
  z_t = \sigma(W_z \cdot [h_{t-1}, x_t] + b_z)
  $$

- **Puerta de reinicio ($r_t$)** decide cuánto del pasado se debe olvidar antes de calcular el nuevo candidato de estado:
  $$
  r_t = \sigma(W_r \cdot [h_{t-1}, x_t] + b_r)
  $$

- **Candidato de estado oculto ($\tilde{h}_t$)** y la actualización del estado oculto:
  $$
  \tilde{h}_t = \tanh(W_h \cdot [r_t \ast h_{t-1}, x_t] + b_h)
  $$
  
  $$
  h_t = (1 - z_t) \ast h_{t-1} + z_t \ast \tilde{h}_t
  $$


#### Ejercicios

1. ¿Qué papel juegan los reguladores como dropout o L2 regularization específicamente en el contexto de RNNs y LSTM para evitar el sobreajuste en tareas de modelado de lenguaje? (1 punto)
2. Considerando la complejidad computacional de BPTT, ¿cuáles son las limitaciones prácticas cuando se usa con RNNs en secuencias muy largas? ¿Cómo podrías mitigar estos problemas en un entorno de producción? (1 punto)


Los reguladores como el **dropout** y la **regularización L2** juegan un papel crucial en el contexto de RNNs y LSTM para evitar el sobreajuste, especialmente en tareas de modelado de lenguaje. A continuación, se describen cómo funcionan estos reguladores y cómo contribuyen a mejorar el rendimiento y la generalización de los modelos.

#### Dropout en RNNs y LSTM

**Dropout** es una técnica de regularización que introduce aleatoriedad en el proceso de entrenamiento, apagando de manera aleatoria un conjunto de neuronas en cada paso de actualización. Esto obliga a la red a aprender representaciones redundantes y robustas, ya que no puede depender de ninguna neurona en particular para una característica específica.

En el contexto de RNNs y LSTM, **dropout** se aplica típicamente de dos maneras:

1. **Dropout en las conexiones no recurrentes**: Esta es la forma más común de aplicar dropout en RNNs. Se aplica dropout en las conexiones de entrada y salida de cada capa recurrente, pero no en las conexiones recurrentes internas. Esto ayuda a prevenir el sobreajuste al asegurarse de que las neuronas no se vuelvan excesivamente dependientes de entradas específicas.
  
   $$
   y_{t} = \text{Dropout}(W_{ih} x_t) + W_{hh} h_{t-1} + b_h
   $$

2. **Variational Dropout**: En LSTM y otras arquitecturas de RNN más complejas, se puede usar una versión modificada de dropout llamada "variational dropout". Aquí, un mismo patrón de apagado se aplica a lo largo de toda la secuencia temporal, en lugar de recalcularse en cada paso de tiempo. Esto mantiene la coherencia en el aprendizaje a través de la secuencia temporal.

#### Regularización L2 en RNNs y LSTM

La **regularización L2** agrega un término de penalización a la función de pérdida del modelo, basado en la suma de los cuadrados de todos los pesos en la red. Esto desalienta que los pesos se vuelvan demasiado grandes, promoviendo soluciones más simples y evitando el sobreajuste.

La regularización L2 se define como:

$$
\mathcal{L}_{total} = \mathcal{L}_{original} + \lambda \sum_{i} w_i^2
$$

Donde $\mathcal{L}_{original}$ es la pérdida original (por ejemplo, error cuadrático medio o entropía cruzada), $w_i$ son los pesos del modelo, y $\lambda$ es el hiperparámetro de regularización que controla la importancia del término de penalización.

#### Efectos y beneficios

- **Dropout**: 
  - Reduce el riesgo de sobreajuste al forzar la red a ser robusta frente a la pérdida de información de neuronas específicas.
  - Mejora la generalización del modelo al hacer que la red no dependa excesivamente de ningún conjunto específico de neuronas.
  - Es particularmente útil en modelos grandes y complejos donde el riesgo de sobreajuste es alto.

- **Regularización L2**:
  - Impide que los pesos del modelo crezcan demasiado, lo que puede llevar a modelos que se ajusten excesivamente a los datos de entrenamiento.
  - Ayuda a mantener los pesos del modelo en un rango más controlado, lo que puede conducir a un aprendizaje más estable y eficiente.
  - Es útil para controlar la complejidad del modelo y mejorar su capacidad de generalización.

#### Aplicación en Tareas de Modelado de Lenguaje

En tareas de modelado de lenguaje, donde los datos son secuenciales y las dependencias a largo plazo son cruciales, el sobreajuste es un problema común. Los reguladores como dropout y L2 son esenciales para:

- **Modelado de lenguaje**: Al entrenar modelos para predecir la siguiente palabra en una secuencia, el dropout puede ayudar a evitar que el modelo se ajuste demasiado a patrones específicos de las secuencias de entrenamiento.
- **Traducción automática**: Las RNNs y LSTMs con dropout y L2 regularization pueden generalizar mejor a frases y contextos nuevos no vistos durante el entrenamiento.
- **Reconocimiento de voz**: El uso de regularización puede mejorar la robustez del modelo frente a diferentes variaciones en las entradas de voz.



La retropropagación a través del tiempo (Backpropagation Through Time, BPTT) es una extensión del algoritmo de retropropagación utilizado para entrenar redes neuronales recurrentes (RNNs). Sin embargo, su aplicación en secuencias muy largas presenta varias limitaciones prácticas debido a su complejidad computacional.

Limitaciones de BPTT en secuencias muy largas


- Explosión del Gradiente: En secuencias largas, los gradientes pueden crecer exponencialmente durante la retropropagación, llevando a actualizaciones de peso grandes e inestables que hacen que el modelo diverja.
- Desaparición del Gradiente: Los gradientes pueden disminuir exponencialmente, haciendo que las actualizaciones de peso sean insignificantes. Esto dificulta el aprendizaje de dependencias a largo plazo.
- La retropropagación a través de largas secuencias requiere almacenar todos los estados ocultos y las intermedias en memoria, lo que puede llevar a un uso intensivo de memoria y recursos computacionales.
- El tiempo de entrenamiento aumenta significativamente con la longitud de la secuencia, lo que puede hacer que el proceso de entrenamiento sea prohibitivo para secuencias muy largas.
- Los cálculos con números muy grandes o muy pequeños pueden provocar inestabilidades numéricas, afectando la precisión y la estabilidad del entrenamiento.

Estrategias para mitigar problemas en un entorno de producción

- En lugar de propagar los gradientes a través de toda la secuencia, se puede truncar BPTT y limitar la propagación a una ventana de longitud fija.
- Esto reduce el uso de memoria y el costo computacional, aunque puede comprometer la capacidad de la red para aprender dependencias a largo plazo.  
     Ejemplo: Si se tiene una secuencia de longitud T, se podría truncar BPTT a una ventana de longitud  k, dondees significativamente menor que 
- Las unidades de memoria de largo y corto plazo (LSTM) y las unidades recurrentes con compuertas (GRU) están diseñadas para manejar dependencias a largo plazo de manera más efectiva que las RNNs simples. Estas arquitecturas incluyen mecanismos internos que ayudan a mitigar los problemas de desaparición y explosión del gradiente.
- Implementar técnicas de regularización como dropout y regularización L2 para evitar el sobreajuste y mejorar la generalización del modelo.
- Regularización de gradiente (Gradient Clipping): Restringir los gradientes a un rango específico para evitar la explosión del gradiente.

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

Batch processing:

- Dividir las secuencias largas en mini-batches para paralelizar el entrenamiento y reducir la carga de memoria.
- Utilizar técnicas como bucketizing para agrupar secuencias de longitud similar y procesarlas en lotes, mejorando la eficiencia computacional.

Optimización del hardware:

- Utilizar hardware especializado como GPUs y TPUs, que están optimizados para operaciones de matriz y pueden manejar mejor los requisitos computacionales de BPTT.
- Implementar operaciones eficientes de memoria y cómputo, aprovechando bibliotecas como CUDA para mejorar el rendimiento.

Modelos alternativos:

- Considerar el uso de arquitecturas más avanzadas como Transformers, que pueden manejar dependencias a largo plazo más eficientemente mediante mecanismos de atención en lugar de recurrencia.
- Los Transformers, como el modelo BERT o GPT, pueden procesar secuencias enteras en paralelo y son menos susceptibles a problemas de gradiente.

In [ ]:
## Parte 3

import torch
from torch import nn
import torch.nn.functional as F

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_hidden = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.tanh(self.input_to_hidden(combined))
        return hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size

        # Gates
        self.input_to_inputgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_forgetgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_outputgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_cellgate = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden, cell):
        combined = torch.cat((input, hidden), 1)

        # Calculate gates
        input_gate = torch.sigmoid(self.input_to_inputgate(combined))
        forget_gate = torch.sigmoid(self.input_to_forgetgate(combined))
        output_gate = torch.sigmoid(self.input_to_outputgate(combined))
        cell_gate = torch.tanh(self.input_to_cellgate(combined))

        # Update cell state
        cell = forget_gate * cell + input_gate * cell_gate
        hidden = output_gate * torch.tanh(cell)

        return hidden, cell

    def init_hidden_and_cell(self):
        return torch.zeros(1, self.hidden_size), torch.zeros(1, self.hidden_size)

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_updategate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_resetgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_newgate = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)

        # Calculate gates
        update_gate = torch.sigmoid(self.input_to_updategate(combined))
        reset_gate = torch.sigmoid(self.input_to_resetgate(combined))
        new_hidden = torch.tanh(self.input_to_newgate(torch.cat((input, reset_gate * hidden), 1)))

        # Update hidden state
        hidden = update_gate * hidden + (1 - update_gate) * new_hidden

        return hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)



Extiende la implementación de LSTM para incluir embeddings de palabras y una capa de clasificación, y entrenar el modelo en una tarea de predicción de la siguiente palabra en secuencias de texto (3 puntos).

- Agrega una capa de embedding al modelo LSTM para procesar entradas de texto.
- Incluye una capa de salida que mapee el estado oculto a las predicciones de palabras.
- Implementa una función de pérdida adecuada para la clasificación de palabras.
- Preprocesa  un corpus de texto grande (utiliza los datos dados en clase por ejemplo) para convertir texto a índices utilizando un vocabulario predefinido.
- Genera datos de entrenamiento como pares de secuencias de entrada y palabras objetivo.

Realiza un análisis de sensibilidad de los hiperparámetros en modelos LSTM y GRU para entender su impacto en la capacidad de aprendizaje de dependencias a largo plazo en textos (3 puntos)

- Selecciona un corpus de texto y prepara datos para el entrenamiento de modelos de lenguaje basados en LSTM y GRU.
- Experimenta con diferentes valores para los hiperparámetros como el tamaño de las puertas, la tasa de aprendizaje, el tamaño del estado oculto y la longitud de BPTT.
- Utiliza técnicas como validación cruzada para evaluar el impacto de estos cambios en la precisión del modelo y en su capacidad para generar texto coherente.
- Analiza cómo la modificación de los parámetros de las puertas y la longitud de BPTT afecta la estabilidad del entrenamiento y la convergencia del modelo.

Para extender la implementación de LSTM y entrenar el modelo en una tarea de predicción de la siguiente palabra en secuencias de texto, necesitamos agregar las siguientes funcionalidades:

- Una capa de embeddings para procesar las entradas de texto.
- Una capa de salida que mapee el estado oculto a las predicciones de palabras.
- Una función de pérdida adecuada para la clasificación de palabras.
- Preprocesar un corpus de texto grande para convertir texto a índices utilizando un vocabulario predefinido.
- Generar datos de entrenamiento como pares de secuencias de entrada y palabras objetivo.


Para extender la implementación de la clase LSTM e incluir funcionalidades adicionales que permitan entrenar el modelo para la tarea de predicción de la siguiente palabra en secuencias de texto usaremos los pasos anteriores. El objetivo es adaptar el modelo para que pueda manejar texto a nivel de palabra, aprendiendo a predecir la siguiente palabra en una secuencia dada. 

1. Agrega una capa de embedding a la clase LSTM que convierta los índices de palabras en vectores densos. Esto permite que el modelo maneje eficazmente el espacio de características de las palabras.

2. Implementa una capa lineal que tome el estado oculto de la LSTM y lo mapee a las predicciones de palabras, es decir, un vector cuyo tamaño sea igual al tamaño del vocabulario.

3. Usara la entropía cruzada, que es común para las tareas de clasificación de múltiples clases en modelos de lenguaje.

4. Desarrollara funciones para convertir un corpus de texto grande en secuencias de índices de palabras utilizando un vocabulario predefinido.

5. Crea pares de secuencias de entrada y palabras objetivo para el entrenamiento.



In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

class LSTMWithEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(LSTMWithEmbedding, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded, hidden)
        logits = self.fc(output[:, -1, :])  # Take the last time step's output
        return logits, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

class TextDataset(Dataset):
    def __init__(self, text, vocab, sequence_length=30):
        self.vocab = vocab
        self.data = self.process_text(text, sequence_length)

    def process_text(self, text, sequence_length):
        tokens = text.split()
        indices = [self.vocab[word] for word in tokens if word in self.vocab]
        data = [(indices[i:i+sequence_length], indices[i+sequence_length])
                for i in range(len(indices) - sequence_length)]
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

def train_model(model, data_loader, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        total_loss = 0
        for x, y in data_loader:
            batch_size = x.size(0)
            hidden = model.init_hidden(batch_size)
            optimizer.zero_grad()
            logits, _ = model(x, hidden)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoca {epoch+1}, Perdida: {total_loss / len(data_loader)}')

# Ejemplo
vocab = {'<UNK>': 0, 'hello': 1, 'world': 2}  
text = "hello world hello world hello"
sequence_length = 2

dataset = TextDataset(text, vocab, sequence_length)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

vocab_size = len(vocab)
embedding_dim = 50
hidden_size = 100
model = LSTMWithEmbedding(vocab_size, embedding_dim, hidden_size)

train_model(model, data_loader, epochs=5)


Realizar un análisis de sensibilidad de los hiperparámetros en modelos LSTM y GRU puede proporcionar información valiosa sobre cómo cada configuración afecta la capacidad del modelo para aprender y predecir secuencias, especialmente en tareas de procesamiento de lenguaje natural como la generación de texto. 

Paso 1: selección y preparación del Corpus de Texto
- Selecciona un corpus de texto adecuado. Por ejemplo, podrías usar un conjunto de datos clásico como el texto de "Alice in Wonderland" o artículos de Wikipedia.
- Tokenización y Vocabulario: Convierte el texto en tokens y construye un vocabulario. Esto incluye convertir cada palabra en un índice numérico para un procesamiento eficiente.
- Crea secuencias de entrada y etiquetas objetivo. Por ejemplo, si usas una longitud de secuencia de 10 palabras, cada secuencia de entrada de 10 palabras debería tener una palabra objetivo que sea la siguiente palabra en el texto.

Implementa dos modelos básicos utilizando LSTM y GRU:



In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

class RNNModel(nn.Module):
    def __init__(self, model_type, vocab_size, embedding_dim, hidden_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if model_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        elif model_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        x, hidden = self.rnn(x, hidden)
        x = self.fc(x[:, -1, :])  
        return x, hidden


Paso 3: Experimentación con hiperparámetros

- Varía el tamaño del estado oculto (por ejemplo, 128, 256, 512).
- Experimenta con diferentes tasas de aprendizaje (por ejemplo, 0.01, 0.001, 0.0001).
- Modifica la longitud de BPTT para ver cómo afecta la capacidad del modelo para aprender dependencias a largo plazo.

Paso 4: Validación cruzada y evaluación

- Divide el conjunto de datos en entrenamiento, validación y pruebas.
-  Utiliza técnicas de validación cruzada para evaluar la generalización del modelo en diferentes subconjuntos del dato.
- Evalúa la precisión de los modelos en la tarea de predicción de palabras y observa la coherencia del texto generado para evaluar la calidad del aprendizaje.

Paso 5: Análisis de resultados
- Observa cómo la variación en la longitud de BPTT y los tamaños de las puertas afecta la estabilidad durante el entrenamiento.
- Analiza cómo cada configuración de hiperparámetros impacta en la capacidad del modelo para capturar y aprender dependencias a largo plazo en el texto.


In [ ]:
def create_model(model_type, vocab_size, embedding_dim, hidden_size):
    model = RNNModel(model_type, vocab_size, embedding_dim, hidden_size)
    return model


Luego, escribiremos un bucle para entrenar modelos con diferentes configuraciones de hiperparámetros:

In [ ]:
hidden_sizes = [128, 256, 512]
learning_rates = [0.01, 0.001, 0.0001]
bptt_lengths = [10, 20, 30]  # Asumiendo que modificamos esto en el diseño de nuestro DataLoader

for hidden_size in hidden_sizes:
    for lr in learning_rates:
        for bptt_length in bptt_lengths:
            model = create_model('LSTM', vocab_size, 100, hidden_size)
            print(f"Entrenamiento con la capa oculta={hidden_size}, lr={lr}, bptt_length={bptt_length}")
            train(model, data_loader, 10, lr)  # Asegúrate de que data_loader use bptt_length


Para realizar una validación cruzada, necesitaríamos implementar un esquema de k-fold o utilizar una división de datos de entrenamiento y validación. Vamos a suponer que se usa una división simple:

In [ ]:
from sklearn.model_selection import train_test_split

# Suponiendo que `dataset` es una instancia de TextDataset
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

def validate(model, data_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in data_loader:
            output, _ = model(x, None)
            loss = criterion(output, y)
            total_loss += loss.item()
    return total_loss / len(data_loader)

# Ejemplo de validación después del entrenamiento
validation_loss = validate(model, val_loader)
print(f'Perdida de la validacion: {validation_loss}')


Analiza cómo los cambios en los hiperparámetros afectan la estabilidad y el rendimiento del modelo:

In [ ]:
results = []
for hidden_size in hidden_sizes:
    for lr in learning_rates:
        for bptt_length in bptt_lengths:
            model = create_model('LSTM', vocab_size, 100, hidden_size)
            train(model, train_loader, 10, lr)
            val_loss = validate(model, val_loader)
            results.append((hidden_size, lr, bptt_length, val_loss))
            print(f"Resultado: Tam oculto={hidden_size}, LR={lr}, BPTT={bptt_length}, Val Loss={val_loss}")

# Analizar los resultados para ver qué configuración da el mejor rendimiento
best_config = min(results, key=lambda x: x[3])
print(f"Mejor configuracion: Hidden Size={best_config[0]}, LR={best_config[1]}, BPTT={best_config[2]}, Loss={best_config[3]}")


### Pregunta 4
El script proporcionado es un ejemplo completo de cómo implementar un modelo de red neuronal recurrente (RNN) utilizando PyTorch para generar texto de manera automática.

In [ ]:
# Importación de librerías necesarias para trabajar con tensores y redes neuronales.
import torch
from torch import nn
import numpy as np

# Datos de entrada: una lista de frases.
text = ['hey how are you','good i am fine','have a nice day']

# Creación de un conjunto de caracteres únicos presentes en las frases.
chars = set(''.join(text))
# Creación de un diccionario que mapea cada caracter a un índice único.
int2char = dict(enumerate(chars))
# Creación de un diccionario inverso que mapea cada índice a su caracter correspondiente.
char2int = {char: ind for ind, char in int2char.items()}

# Determinación de la longitud máxima de las frases para normalizar la longitud de todas.
maxlen = len(max(text, key=len))
print("La longitud mayor tiene {} caracteres".format(maxlen))

# Añadir espacios a las frases más cortas para igualar la longitud máxima.
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

# Inicialización de listas para secuencias de entrada y objetivo.
input_seq = []
target_seq = []

# Creación de secuencias de entrada y objetivo.
for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
    print("Secuencia entrada: {}\nSecuencia objetivo: {}".format(input_seq[i], target_seq[i]))

# Conversión de caracteres a índices para procesamiento numérico.
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

# Definición de tamaños para la codificación one-hot.
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

# Función para codificar las secuencias en formato one-hot.
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
          features[i, u, sequence[i][u]] = 1
    return features

# Aplicación de la codificación one-hot a las secuencias de entrada.
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Forma de entrada: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

# Conversión de las secuencias de entrada a tensores de PyTorch.
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

# Chequeo de disponibilidad de GPU y selección del dispositivo (GPU o CPU).
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU es disponible")
else:
    device = torch.device("cpu")
    print("GPU no disponible, CPU es usada")

# Definición de la clase del modelo RNN.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con parámetros específicos.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.
  
  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)
    
  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval() 
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')


#### Ejercicios: 

1. Modifica el modelo existente para que funcione como un autoencoder. Esto implica que el modelo debe aprender a codificar una secuencia de entrada en un vector de características (estado oculto) y luego decodificar ese vector de vuelta a la secuencia original (1.5 puntos).
     - Implementa las capas de codificación y decodificación dentro del mismo modelo.
     - Experimenta  con diferentes estructuras como LSTM para mejorar la retención de información.
     - Mide la calidad de la reconstrucción del texto y la eficiencia de compresión.

2. Utiliza el modelo RNN actual y modifícalo para introducir secuencias más largas. Monitoriza los gradientes durante el entrenamiento para detectar signos de desaparición o explosión. (1.5 puntos)
    - Implementa el  clipping de gradiente para prevenir la explosión del gradiente.
    - Reemplaza la RNN por LSTM para abordar la desaparición del gradiente.
    - Utiliza técnicas de visualización para observar la magnitud de los gradientes a lo largo de varias épocas.
3. Implementa el dropout en las capas recurrentes y comparar los resultados. (1 punto)

    - Ajusta el parámetro de weight decay en el optimizador y observar el efecto sobre el overfitting.
    - Aplica early stopping basado en la validación del loss para detener el entrenamiento antes de que el modelo comience a sobreajustarse.


Para modificar el modelo RNN existente y convertirlo en un autoencoder que aprenda a codificar una secuencia de entrada en un vector de características (estado oculto) y luego decodificar ese vector de vuelta a la secuencia original, se necesitará realizar varias modificaciones estructurales y funcionales en el modelo. 

1. Definición del modelo Autoencoder RNN

La estructura básica del autoencoder RNN incluirá capas de codificación y decodificación. Usaremos LSTM para la capa de codificación para capturar dependencias a largo plazo, y otra red LSTM o RNN para la decodificación.

In [ ]:
class RNN_Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers):
        super(RNN_Autoencoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Codificador
        self.encoder = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        
        # Decodificador
        self.decoder = nn.LSTM(hidden_dim, input_size, n_layers, batch_first=True)  # Podría ser también un RNN simple o GRU

    def forward(self, x):
        batch_size = x.size(0)

        # Codificación
        _, (hidden, _) = self.encoder(x)

        # Decodificación
        # Repetir el vector de características a través de la secuencia para decodificar
        repeat_hidden = hidden.repeat(1, x.size(1), 1)
        output, _ = self.decoder(repeat_hidden)

        return output

    def init_hidden(self, batch_size):
        return torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)


2 . Instanciación del modelo y configuración de entrenamiento

Configura el autoencoder y define los hiperparámetros de entrenamiento.

In [ ]:
# Hiperparámetros
input_size = dict_size
hidden_dim = 12
n_layers = 1

# Instancia del modelo
autoencoder = RNN_Autoencoder(input_size, hidden_dim, n_layers)
autoencoder.to(device)

# Configuración de entrenamiento
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.01)

# Entrenamiento del modelo
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad()
    output = autoencoder(input_seq)
    loss = criterion(output, input_seq)  # Ahora se compara la salida con la entrada
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoca: {epoch}/{n_epochs} Perdida: {loss.item():.4f}')


3 . Prueba y evaluación

Evaluaremos el modelo autoencoder en términos de calidad de la reconstrucción y la eficiencia de la compresión del texto.

In [ ]:
# Función de evaluación
def evaluate_autoencoder(model, input_seq):
    model.eval()
    with torch.no_grad():
        output = model(input_seq)
    return output

# Convertir el tensor de salida en texto para verificar la reconstrucción
output_seq = evaluate_autoencoder(autoencoder, input_seq)
reconstructed_text = torch.argmax(output_seq, dim=2)
reconstructed_text = ''.join([int2char[int(idx)] for idx in reconstructed_text[0]])

print(f'Texto reconstruido: {reconstructed_text}')


Modificar el modelo RNN para manejar secuencias más largas y monitorizar los gradientes durante el entrenamiento es una excelente forma de mejorar la robustez del modelo y prevenir problemas comunes como la desaparición y explosión de gradientes. Implementamos estos cambios utilizando PyTorch, introduciendo clipping de gradiente, sustituyendo RNN por LSTM, y utilizando técnicas de visualización para los gradientes.

1 . Reemplazar RNN por LSTM

Primero, reemplacemos la capa RNN por una LSTM para mejorar la capacidad del modelo para aprender dependencias a largo plazo y abordar la desaparición del gradiente:

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Capa LSTM
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])  # Tomamos la última salida temporal para la predicción
        return out
    
    def init_hidden(self, batch_size):
        # Inicializar el estado oculto y el estado de la celda a cero
        hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device),
                  torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device))
        return hidden


2 . Implementación del clipping de gradiente

Añade clipping de gradiente en el bucle de entrenamiento para prevenir la explosión del gradiente. Esto limita la norma del gradiente a un valor máximo deseado:

In [ ]:
# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01
clip=5  # Valor máximo para la norma de los gradientes

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad()
    output = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward()
    
    # Aplicar clipping de gradiente
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    
    optimizer.step()
    
    if epoch%10 == 0:
        print('Epoca: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Perdida: {:.4f}".format(loss.item()))


3 . Técnicas de Visualización para los Gradientes

Usamos matplotlib para visualizar la magnitud de los gradientes puede ayudarte a entender cómo evolucionan durante el entrenamiento:



In [ ]:
import matplotlib.pyplot as plt

# Guardar magnitudes de gradiente para visualización
grad_magnitudes = []

def plot_grad_flow(named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
    grad_magnitudes.append(ave_grads)

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad()
    output = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward()
    plot_grad_flow(model.named_parameters())
    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()

# Plotting
plt.figure(figsize=(10,5))
plt.imshow(np.array(grad_magnitudes).T, cmap='hot', interpolation='nearest', aspect='auto')
plt.colorbar()
plt.yticks(range(len(layers)), layers)
plt.title("Gradiente Magnitud a lo largo de Épocas")
plt.xlabel("Época")
plt.ylabel("Capas")
plt.show()


Incorporar técnicas como dropout en las capas recurrentes, ajustar el parámetro de weight decay en el optimizador y aplicar early stopping son métodos efectivos para mejorar la generalización del modelo y prevenir el sobreajuste. A continuación, te muestro cómo implementar estos métodos en el contexto de un modelo LSTM en PyTorch.

1 . Implementación del Dropout en las capas recurrentes

Modificar el modelo para incluir dropout en las capas LSTM. Esto ayuda a prevenir el sobreajuste al introducir regularización en el modelo.

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout_prob):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # LSTM con dropout
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, batch_size):
        # Inicializar el estado oculto y el estado de la celda a cero
        hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device),
                  torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device))
        return hidden

model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=2, dropout_prob=0.5)


2 . Ajuste del parámetro de weight decay en el optimizador

Configurar el optimizador para usar weight decay, lo cual añade una penalización L2 a los pesos durante el entrenamiento, ayudando a reducir el sobreajuste.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)  # Ajuste de weight decay

3 . Aplicación de early stopping

Implementar early stopping para finalizar el entrenamiento si el loss de validación no mejora después de un número determinado de épocas. Esto evita que el modelo sobreajuste al conjunto de datos de entrenamiento.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'Contador EarlyStopping : {self.counter} de {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# Ejemplo de uso de EarlyStopping
early_stopping = EarlyStopping(patience=10, verbose=True)

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad()
    output = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward()
    optimizer.step()

    val_output = model(val_seq)  # Suponiendo que val_seq es la secuencia de validación
    val_loss = criterion(val_output, val_target_seq.view(-1).long())

    print(f'Epoca {epoch}, Perdida de entrenamiento: {loss.item()}, Perdida de validacion: {val_loss.item()}')

    early_stopping(val_loss.item(), model)
    if early_stopping.early_stop:
        print("Early stopping")
        break


Estos métodos proporcionan maneras robustas de mejorar la generalización del modelo y prevenir el sobreajuste. Implementando dropout en las capas LSTM, ajustando weight decay en el optimizador y utilizando early stopping basado en el desempeño en el conjunto de validación, podemos obtener un modelo más robusto y efectivo para tareas de aprendizaje secuencial.